In [1]:
import src.requestcompletion as rc
from src.requestcompletion.llm import MessageHistory, SystemMessage, UserMessage
from src.requestcompletion.nodes.library import from_function
from src.requestcompletion.visuals.agent_viewer import AgentViewer
from pydantic import BaseModel

# Voice Transcription Agent Demo

We will make a voice to notion agent that can transcribe your voice commands.

Then we will wrap this agent in a top level agent that will call the voice agent and pass the voice user prompt to the notion agent.

### Step 1: Creating the VoiceAgent

In [2]:
# Importing all functions that the notion_agent can invoke
from demo.sample_tools.audio_tools import record_audio, transcribe_audio

We have already created the NotionAgent, ImageAgent, and AudioAgent in the sample_agents folder. Instead of importing and calling them directly, we will make the VoiceAgent to demonstrate how easy it is to use the RC library.

In [3]:
class VoiceTranscription(BaseModel):
    user_voice_command: str

SYSTEM_PROMPT = SystemMessage(
    """
    You are a helpful assistant that can record users audio and transcribe it.
    """
)

AudioAgent = rc.library.tool_call_llm(
    pretty_name="Audio Agent",
    system_message=SYSTEM_PROMPT,
    model=rc.llm.OpenAILLM(model_name="gpt-4o"),
    connected_nodes={
        from_function(record_audio),
        from_function(transcribe_audio),
    },
    output_model=VoiceTranscription,        # for structured output with tool_call_llm
)

Yes, that is all the code you need to create a tool_calling VoiceAgent 😙

### Step 2: Importing the NotionAgent

In [4]:
# We made a few sample_agents
from demo.sample_agents import NotionAgent   

### Step 3: Wrap both agents in a top level function and create a node from the function

In [5]:
async def top_level_node(user_prompt: str):
    voice_transcription = await rc.call(AudioAgent, 
                           message_history=MessageHistory([UserMessage(user_prompt)]))
    
    await rc.call(NotionAgent, message_history=MessageHistory([UserMessage(voice_transcription.user_voice_command)]))
    
VoiceNotionAgent = from_function(top_level_node)

### Step 4: Invoke the top level node

In [6]:
USER_PROMPT = "Record my audio for 10 seconds and transcribe it."

In [7]:
with rc.Runner(executor_config=rc.ExecutorConfig(timeout=50)) as runner:
    result = await runner.run(VoiceNotionAgent, 
                              user_prompt=USER_PROMPT,
                              )

[+3.350  s] RC.RUNNER   : INFO     - START CREATED top_level_node Node - (, user_prompt=Record my audio for 10 seconds and transcribe it.)
[+3.351  s] RC.RUNNER   : INFO     - top_level_node Node CREATED Audio Agent - (, message_history=user: Record my audio for 10 seconds and transcribe it.)
[+5.164  s] RC.RUNNER   : INFO     - Audio Agent CREATED record_audio Node - ({'filename': 'user_audio.wav', 'duration': 10}, )


Starting 10 second recording...
Recording complete. Saving file...


[+16.817 s] RC.RUNNER   : INFO     - record_audio Node DONE None


Saved to user_audio.wav


[+17.484 s] RC.RUNNER   : INFO     - Audio Agent CREATED transcribe_audio Node - ({'filename': 'user_audio.wav'}, )
[+19.165 s] RC.RUNNER   : INFO     - transcribe_audio Node DONE create a page called test1 under agent demo root
[+19.832 s] RC.RUNNER   : INFO     - Audio Agent CREATED VoiceTranscription - (, message_history=user: The transcribed audio is: "create a page called test1 under agent demo root.")
[+20.582 s] RC.RUNNER   : INFO     - VoiceTranscription DONE user_voice_command='create a page called test1 under agent demo root.'
[+20.582 s] RC.RUNNER   : INFO     - Audio Agent DONE user_voice_command='create a page called test1 under agent demo root.'
[+20.583 s] RC.RUNNER   : INFO     - top_level_node Node CREATED Notion Agent - (, message_history=user: create a page called test1 under agent demo root.)
[+21.434 s] RC.RUNNER   : INFO     - Notion Agent CREATED find_page Node - ({'query': 'agent demo root'}, )
[+21.784 s] RC.RUNNER   : INFO     - find_page Node DONE [PageProper

### Additional: Graphical representation of the nodes and the flow of data

In [8]:
viewer = AgentViewer(
        stamps=result.all_stamps,
        request_heap=result.request_heap,
        node_heap=result.node_heap
    )
viewer.display_graph()